In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests

In [2]:
# 72 = WWC, 37 = FA WSL, 43 = MWC, 49 = NWSL
competition = 'WWC'
compID = 72

In [3]:
match_info_url = "https://raw.githubusercontent.com/statsbomb/open-data/master/data/matches/"+str(compID)+".json"

# Pull data from url into python dictionary
games = requests.get(url=match_info_url).json()

# Use list comprehension to create a list of match ids
match_ids = [g['match_id'] for g in games]


print("There are {} games in this list".format(len(match_ids)))

There are 52 games in this list


In [4]:
MatchData = [] 

for p in games:
    attributes = {
        "Home": p['home_team']['home_team_name'],
        "Away": p['away_team']['away_team_name'],
        "Competition": p['competition']['competition_name'],
        "Home Score": p['home_score'],
        "Away Score": p['away_score'],
        "Date": p['match_date'],
        "Match ID": p['match_id']

    }

    MatchData.append(attributes)
    
Matches = pd.DataFrame(data=MatchData, columns=['Competition', 'Date', 'Match ID','Home', 'Home Score', 'Away', 'Away Score'])
Matches.head()

,Competition,Date,Match ID,Home,Home Score,Away,Away Score
0,Women's World Cup,2019-06-18,68311,Jamaica Women's,1,Australia Women's,4
1,Women's World Cup,2019-06-18,68312,Italy Women's,0,Brazil Women's,1
2,Women's World Cup,2019-06-19,68337,Japan Women's,0,England Women's,2
3,Women's World Cup,2019-06-19,68338,Scotland Women's,3,Argentina Women's,3
4,Women's World Cup,2019-06-20,68343,Thailand Women's,0,Chile Women's,2
5,Women's World Cup,2019-06-20,68344,Netherlands Women's,2,Canada Women's,1
6,Women's World Cup,2019-06-20,68345,Sweden Women's,0,United States Women's,2
7,Women's World Cup,2019-06-20,68346,Cameroon Women's,2,New Zealand Women's,1
8,Women's World Cup,2019-06-22,68355,Germany Women's,3,Nigeria Women's,0
9,Women's World Cup,2019-06-23,68362,England Women's,3,Cameroon Women's,0


In [5]:
Matches.to_csv('/Users/jmuwsoccer/Documents/Python/CSV/'+str(competition)+' Matches.csv')

In [6]:
match_ids = []
for g in games:
    m_id = g['match_id']
    match_ids.append(m_id)

In [7]:
PassData = []

# template URL at which each game's event data is stored
match_data_url = "https://raw.githubusercontent.com/statsbomb/open-data/master/data/events/{}.json"

# This takes some time, naturally
for match_id in match_ids:
    
    # Get all events from a particular game
    all_events = requests.get(url=match_data_url.format(match_id)).json()
        
    # Filter them by type-name Pass
    passes = [x for x in all_events if x['type']['name'] == "Pass"]
    
    for p in passes:
        attributes = {
            "Player": p['player']['name'],
            "Team": p['team']['name'],
            "X": p['location'][0],
            "Y": p['location'][1],
            "DestX": p['pass']['end_location'][0],
            "DestY": p['pass']['end_location'][1],
            "Length": p['pass']['length'],
            "Angle": p['pass']['angle'],
            # Multiple other features can be collected here, such as pressure
            "ifComplete": 0 if "outcome" in p['pass'] else 1,
            "PassesAttempted": 1 if "outcome" in p['pass'] else 1,
            "Height": p['pass']['height']['name'],
            "Position": p['position']['name'],
            "Type of Attack": p['play_pattern']['name'],
            "Minute": p['minute'],
            "Second": p['second'],
            "Possession": p['possession'],
            "Type": p['type']['name'],
            "Half": p['period'],
            "Recipient": p['pass']['recipient']['name'] if "recipient" in p['pass'] else "None",
            "UnderPressure": 1 if "under_pressure" in p else 0,
            "ifAssist": 1 if "goal_assist" in p['pass'] else 0,
            "ThroughBall": 1 if "through_ball" in p['pass'] else 0,
            "MatchID": match_id,
            "CompetitionID": compID
        }
        
        PassData.append(attributes)

In [10]:
PassData = pd.DataFrame(data=PassData, columns=['CompetitionID','MatchID','Team', 'Player', 'Position', 'ifComplete', 'ifAssist', 'X', 'Y', 'DestX', 'DestY', 'Height', 'Angle', 'Length', 'Type of Attack', 'Possession', 'PassesAttempted', 'Half','Recipient', "ThroughBall", 'UnderPressure', 'Minute', 'Second'])
PassData.head()

,CompetitionID,MatchID,Team,Player,Position,ifComplete,ifAssist,X,Y,DestX,...,Length,Type of Attack,Possession,PassesAttempted,Half,Recipient,ThroughBall,UnderPressure,Minute,Second
0,72,68311,Jamaica Women's,Khadija Monifa Shaw,Center Attacking Midfield,1,0,60.0,40.0,55.0,...,7.856844,From Kick Off,2,1,1,Toriana Jade Patterson,0,0,0,1
1,72,68311,Jamaica Women's,Toriana Jade Patterson,Left Center Midfield,1,0,55.0,36.0,99.0,...,58.062122,From Kick Off,2,1,1,Mireya Grey,0,0,0,3
2,72,68311,Jamaica Women's,Mireya Grey,Right Wing,1,0,110.0,73.0,108.0,...,9.144397,From Kick Off,2,1,1,Cheyna Matthews,0,0,0,10
3,72,68311,Jamaica Women's,Cheyna Matthews,Center Forward,0,0,108.0,65.0,114.0,...,6.300000,From Kick Off,2,1,1,Mireya Grey,0,0,0,12
4,72,68311,Australia Women's,Lydia Williams,Goalkeeper,1,0,6.0,33.0,14.0,...,18.987364,From Free Kick,3,1,1,Alanna Kennedy,0,0,0,22


In [11]:
PassData['ifGround'] = [1 if x == 'Ground Pass' else 0 for x in PassData['Height']]
PassData['ifLow'] = [1 if x == 'Low Pass' else 0 for x in PassData['Height']]
PassData['ifHigh'] = [1 if x == 'High Pass' else 0 for x in PassData['Height']]
PassData['DistX'] = PassData['DestX'] - PassData['X']
PassData['PP'] = [1 if x >= 23 else 0 for x in PassData['DistX']]

In [12]:
PassData['IntoF3'] = [1 if x >= 80 else 0 for x in PassData['DestX']]
PassData['From23'] = [1 if x <= 79 else 0 for x in PassData['X']]
PassData['DPSum'] = PassData['IntoF3'] + PassData['From23']
PassData['DP'] = [1 if x == 2 else 0 for x in PassData['DPSum']]

In [13]:
PassData = pd.DataFrame(data=PassData, columns=['CompetitionID','MatchID','Team', 'Player', 'Position', 'ifComplete', 'ifAssist', 'X', 'Y', 'DestX', 'DestY', 'Height', 'Angle', 'Length', 'Type of Attack', 'Possession', 'Minute', 'Second', 'PassesAttempted', 'Half','Recipient', 'ThroughBall', 'UnderPressure', 'PP', 'DP', 'ifGround', 'ifLow', 'ifHigh'])
PassData.head()

,CompetitionID,MatchID,Team,Player,Position,ifComplete,ifAssist,X,Y,DestX,...,PassesAttempted,Half,Recipient,ThroughBall,UnderPressure,PP,DP,ifGround,ifLow,ifHigh
0,72,68311,Jamaica Women's,Khadija Monifa Shaw,Center Attacking Midfield,1,0,60.0,40.0,55.0,...,1,1,Toriana Jade Patterson,0,0,0,0,1,0,0
1,72,68311,Jamaica Women's,Toriana Jade Patterson,Left Center Midfield,1,0,55.0,36.0,99.0,...,1,1,Mireya Grey,0,0,1,1,0,0,1
2,72,68311,Jamaica Women's,Mireya Grey,Right Wing,1,0,110.0,73.0,108.0,...,1,1,Cheyna Matthews,0,0,0,0,1,0,0
3,72,68311,Jamaica Women's,Cheyna Matthews,Center Forward,0,0,108.0,65.0,114.0,...,1,1,Mireya Grey,0,0,0,0,1,0,0
4,72,68311,Australia Women's,Lydia Williams,Goalkeeper,1,0,6.0,33.0,14.0,...,1,1,Alanna Kennedy,0,0,0,0,1,0,0


In [14]:
PassData.to_csv('/Users/jmuwsoccer/Documents/Python/CSV/'+str(competition)+' Passes.csv')

In [15]:
DribbleData = []

# template URL at which each game's event data is stored
match_data_url = "https://raw.githubusercontent.com/statsbomb/open-data/master/data/events/{}.json"

# This takes some time, naturally
for match_id in match_ids:
    
    # Get all events from a particular game
    all_events = requests.get(url=match_data_url.format(match_id)).json()
        
    # Filter them by type-name Pass
    dribble = [x for x in all_events if x['type']['name'] == "Dribble"]
    
    for p in dribble:
        attributes = {
            "Player": p['player']['name'],
            "Team": p['team']['name'],
            "X": p['location'][0],
            "Y": p['location'][1],
            # Multiple other features can be collected here, such as pressure
            "Outcome":p['dribble']['outcome']['name'],
            "Position": p['position']['name'],
            "Type of Attack": p['play_pattern']['name'],
            "Minute": p['minute'],
            "Second": p['second'],
            "Possession": p['possession'],
            "Type": p['type']['name'],
            "Half": p['period'],
            "UnderPressure": p['under_pressure'],
            "MatchID": match_id,
            "CompetitionID": compID
        }
        
        DribbleData.append(attributes)

In [16]:
DribbleData = pd.DataFrame(data=DribbleData, columns=['CompetitionID','MatchID','Team', 'Player', 'Position', 'X', 'Y', 'Type of Attack', 'Outcome', 'Possession', 'Minute', 'Second', 'Half'])
DribbleData.head()

,CompetitionID,MatchID,Team,Player,Position,X,Y,Type of Attack,Outcome,Possession,Minute,Second,Half
0,72,68311,Jamaica Women's,Mireya Grey,Right Wing,112.0,77.0,From Kick Off,Complete,2,0,9,1
1,72,68311,Jamaica Women's,Tiffany Cameron,Left Wing,30.0,16.0,Regular Play,Incomplete,6,2,6,1
2,72,68311,Jamaica Women's,Mireya Grey,Right Wing,38.0,77.0,From Throw In,Complete,8,3,26,1
3,72,68311,Australia Women's,Samantha Kerr,Center Forward,118.0,56.0,From Counter,Incomplete,9,3,55,1
4,72,68311,Jamaica Women's,Deneisha Selena Blackwood,Left Back,43.0,4.0,Regular Play,Complete,10,4,6,1


In [17]:
DribbleData.to_csv('/Users/jmuwsoccer/Documents/Python/CSV/'+str(competition)+' Dribbles.csv')

In [18]:
ShotData = []

# template URL at which each game's event data is stored
match_data_url = "https://raw.githubusercontent.com/statsbomb/open-data/master/data/events/{}.json"

# This takes some time, naturally
for match_id in match_ids:
    
    # Get all events from a particular game
    all_events = requests.get(url=match_data_url.format(match_id)).json()
        
    # Filter them by type-name Pass
    shots = [x for x in all_events if x['type']['name'] == "Shot"]
    
    for s in shots:
        attributes = {
            "Player": s['player']['name'],
            "Team": s['team']['name'],
            "X": s['location'][0],
            "Y": s['location'][1],
            "xG": s['shot']['statsbomb_xg'],
            "Position": s['position']['name'],
            "Possession": s['possession'],
            "Type of Attack": s['play_pattern']['name'],
            "Body Part": s['shot']['body_part']['name'],
            "Action": s['shot']['outcome']['name'] if "outcome" in s['shot'] else "Off T",
            "Technique":s['shot']['technique']['name'],
            "OPorFK": s['shot']['type']['name'] if "type" in s['shot'] else 0,
            "Half": s['period'],
            "Minute": s['minute'],
            "Second": s['second'],
            "MatchID": match_id,
            "CompetitionID": compID
            # Multiple other features can be collected here, such as pressure
           
        }
        
        ShotData.append(attributes)

In [19]:
ShotData = pd.DataFrame(data=ShotData, columns=['CompetitionID','MatchID', 'Possession','Team', 'Player', 'Position', 'xG', 'Action', 'Type of Attack', 'OPorFK','Body Part', 'X', 'Y', 'Technique', 'Half', 'Minute', 'Second'])
ShotData.head()

,CompetitionID,MatchID,Possession,Team,Player,Position,xG,Action,Type of Attack,OPorFK,Body Part,X,Y,Technique,Half,Minute,Second
0,72,68311,12,Australia Women's,Samantha Kerr,Center Forward,0.202099,Saved,Regular Play,Open Play,Right Foot,104.0,47.0,Normal,1,4,49
1,72,68311,25,Australia Women's,Samantha Kerr,Center Forward,0.019987,Goal,From Free Kick,Open Play,Head,108.0,38.0,Normal,1,10,14
2,72,68311,38,Australia Women's,Samantha Kerr,Center Forward,0.031038,Off T,Regular Play,Open Play,Head,109.0,49.0,Normal,1,18,12
3,72,68311,48,Australia Women's,Samantha Kerr,Center Forward,0.015194,Saved,From Throw In,Open Play,Head,106.0,44.0,Normal,1,24,52
4,72,68311,49,Australia Women's,Samantha Kerr,Center Forward,0.050592,Off T,From Corner,Open Play,Head,113.0,43.0,Normal,1,25,27


In [20]:
ShotData['ifGoal'] = [1 if x == 'Goal' else 0 for x in ShotData['Action']]
ShotData['ifPen'] = [1 if x == 'Penalty' else 0 for x in ShotData['OPorFK']]
ShotData['Shots'] = [1 if x >= 0 else 0 for x in ShotData['xG']]
ShotData.head()

,CompetitionID,MatchID,Possession,Team,Player,Position,xG,Action,Type of Attack,OPorFK,Body Part,X,Y,Technique,Half,Minute,Second,ifGoal,ifPen,Shots
0,72,68311,12,Australia Women's,Samantha Kerr,Center Forward,0.202099,Saved,Regular Play,Open Play,Right Foot,104.0,47.0,Normal,1,4,49,0,0,1
1,72,68311,25,Australia Women's,Samantha Kerr,Center Forward,0.019987,Goal,From Free Kick,Open Play,Head,108.0,38.0,Normal,1,10,14,1,0,1
2,72,68311,38,Australia Women's,Samantha Kerr,Center Forward,0.031038,Off T,Regular Play,Open Play,Head,109.0,49.0,Normal,1,18,12,0,0,1
3,72,68311,48,Australia Women's,Samantha Kerr,Center Forward,0.015194,Saved,From Throw In,Open Play,Head,106.0,44.0,Normal,1,24,52,0,0,1
4,72,68311,49,Australia Women's,Samantha Kerr,Center Forward,0.050592,Off T,From Corner,Open Play,Head,113.0,43.0,Normal,1,25,27,0,0,1


In [22]:
OGFData = []

# template URL at which each game's event data is stored
match_data_url = "https://raw.githubusercontent.com/statsbomb/open-data/master/data/events/{}.json"

# This takes some time, naturally
for match_id in match_ids:
    
    # Get all events from a particular game
    all_events = requests.get(url=match_data_url.format(match_id)).json()
        
    # Filter them by type-name Pass
    OGF = [x for x in all_events if x['type']['name'] == "Own Goal For"]
    
    
    for s in OGF:
        attributes = {
            "Team": s['team']['name'],
            "Action": s['type']['name'],
            "Half": s['period'],
            "Minute": s['minute'],
            "Second": s['second'],
            "MatchID": match_id,
            "Possession": s['possession'],
            "CompetitionID": compID
            # Multiple other features can be collected here, such as pressure
        }
        
        OGFData.append(attributes)

In [23]:
OGFData = pd.DataFrame(data=OGFData, columns=['CompetitionID','MatchID', 'Possession', 'Team', 'Player', 'Position', 'xG', 'Action', 'Type of Attack', 'OPorFK','Body Part', 'X', 'Y', 'Technique', 'Half', 'Minute', 'Second',])
OGFData.head()

,CompetitionID,MatchID,Possession,Team,Player,Position,xG,Action,Type of Attack,OPorFK,Body Part,X,Y,Technique,Half,Minute,Second
0,72,68338,165,Argentina Women's,NaN,NaN,NaN,Own Goal For,NaN,NaN,NaN,NaN,NaN,NaN,2,78,32
1,72,68343,114,Chile Women's,NaN,NaN,NaN,Own Goal For,NaN,NaN,NaN,NaN,NaN,NaN,2,47,31
2,72,68345,106,United States Women's,NaN,NaN,NaN,Own Goal For,NaN,NaN,NaN,NaN,NaN,NaN,2,49,16
3,72,68346,177,New Zealand Women's,NaN,NaN,NaN,Own Goal For,NaN,NaN,NaN,NaN,NaN,NaN,2,79,1
4,72,22924,81,Norway Women's,NaN,NaN,NaN,Own Goal For,NaN,NaN,NaN,NaN,NaN,NaN,1,36,23


In [24]:
OGFData['ifGoal'] = [1 if x == 'Own Goal For' else 0 for x in OGFData['Action']]
OGFData['Shots'] = [0 if x == 'Own Goal For' else 0 for x in OGFData['Action']]
OGFData = OGFData.replace(to_replace = np.nan, value =0) 

In [25]:
frames = [ShotData, OGFData]
ShotData = pd.concat(frames)
ShotData = pd.DataFrame(data=ShotData, columns=['CompetitionID','MatchID', 'Possession', 'Team', 'Player', 'Position', 'Action', 'xG', 'ifGoal', 'Shots', 'ifPen', 'Type of Attack', 'OPorFK','Body Part', 'X', 'Y', 'Technique', 'Half', 'Minute', 'Second'])
ShotData.head()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


,CompetitionID,MatchID,Possession,Team,Player,Position,Action,xG,ifGoal,Shots,ifPen,Type of Attack,OPorFK,Body Part,X,Y,Technique,Half,Minute,Second
0,72,68311,12,Australia Women's,Samantha Kerr,Center Forward,Saved,0.202099,0,1,0.0,Regular Play,Open Play,Right Foot,104.0,47.0,Normal,1,4,49
1,72,68311,25,Australia Women's,Samantha Kerr,Center Forward,Goal,0.019987,1,1,0.0,From Free Kick,Open Play,Head,108.0,38.0,Normal,1,10,14
2,72,68311,38,Australia Women's,Samantha Kerr,Center Forward,Off T,0.031038,0,1,0.0,Regular Play,Open Play,Head,109.0,49.0,Normal,1,18,12
3,72,68311,48,Australia Women's,Samantha Kerr,Center Forward,Saved,0.015194,0,1,0.0,From Throw In,Open Play,Head,106.0,44.0,Normal,1,24,52
4,72,68311,49,Australia Women's,Samantha Kerr,Center Forward,Off T,0.050592,0,1,0.0,From Corner,Open Play,Head,113.0,43.0,Normal,1,25,27


In [26]:
ShotData.to_csv('/Users/jmuwsoccer/Documents/Python/CSV/'+str(competition)+' Shots.csv')

In [27]:
PressureData = []

# template URL at which each game's event data is stored
match_data_url = "https://raw.githubusercontent.com/statsbomb/open-data/master/data/events/{}.json"

# This takes some time, naturally
for match_id in match_ids:
    
    # Get all events from a particular game
    all_events = requests.get(url=match_data_url.format(match_id)).json()
        
    # Filter them by type-name Pass
    pressures = [x for x in all_events if x['type']['name'] == "Pressure"]
    
    for p in pressures:
        attributes = {
            "Player": p['player']['name'],
            "Team": p['team']['name'],
            "Position": p['position']['name'],
            "X": p['location'][0],
            "Y": p['location'][1],
            "Type of Attack": p['play_pattern']['name'],
            "Half": p['period'],
            "Minute": p['minute'],
            "Second": p['second'],
            "Duration": p['duration'] if "duration" in p else 0,
            "MatchID": match_id,
            "CompetitionID": compID
        }
        
        PressureData.append(attributes)

In [28]:
PressureData = pd.DataFrame(data=PressureData, columns=['CompetitionID','MatchID','Team', 'Player', 'Position', 'Type of Attack', 'X', 'Y', 'Half', 'Minute','Second','Duration'])
PressureData.head()

,CompetitionID,MatchID,Team,Player,Position,Type of Attack,X,Y,Half,Minute,Second,Duration
0,72,68311,Australia Women's,Emily Gielnik,Right Wing,From Kick Off,64.0,47.0,1,0,2,0.759983
1,72,68311,Australia Women's,Karly Roestbakken,Left Back,From Kick Off,18.0,8.0,1,0,7,2.166599
2,72,68311,Australia Women's,Karly Roestbakken,Left Back,From Kick Off,6.0,17.0,1,0,13,0.646988
3,72,68311,Jamaica Women's,Toriana Jade Patterson,Left Center Midfield,From Free Kick,48.0,64.0,1,0,41,0.967883
4,72,68311,Australia Women's,Lisa De Vanna,Left Wing,Regular Play,73.0,19.0,1,0,43,0.617694


In [29]:
PressureData['StartSeconds'] = PressureData['Minute']*60 + PressureData['Second'] - 5
PressureData['EndSeconds'] = (PressureData['Minute']*60 + PressureData['Second']) + PressureData['Duration'] + 15

In [30]:
PressureData = pd.DataFrame(data=PressureData, columns=['CompetitionID','MatchID','Team', 'Player', 'Position', 'Type of Attack', 'X', 'Y', 'Half', 'StartSeconds', 'EndSeconds'])
PressureData.head()

,CompetitionID,MatchID,Team,Player,Position,Type of Attack,X,Y,Half,StartSeconds,EndSeconds
0,72,68311,Australia Women's,Emily Gielnik,Right Wing,From Kick Off,64.0,47.0,1,-3,17.759983
1,72,68311,Australia Women's,Karly Roestbakken,Left Back,From Kick Off,18.0,8.0,1,2,24.166599
2,72,68311,Australia Women's,Karly Roestbakken,Left Back,From Kick Off,6.0,17.0,1,8,28.646988
3,72,68311,Jamaica Women's,Toriana Jade Patterson,Left Center Midfield,From Free Kick,48.0,64.0,1,36,56.967883
4,72,68311,Australia Women's,Lisa De Vanna,Left Wing,Regular Play,73.0,19.0,1,38,58.617694


In [31]:
PressureData.to_csv('/Users/jmuwsoccer/Documents/Python/CSV/'+str(competition)+' Pressure.csv')

In [32]:
DuelData = []

# template URL at which each game's event data is stored
match_data_url = "https://raw.githubusercontent.com/statsbomb/open-data/master/data/events/{}.json"

# This takes some time, naturally
for match_id in match_ids:
    
    # Get all events from a particular game
    all_events = requests.get(url=match_data_url.format(match_id)).json()
        
    # Filter them by type-name Pass
    pressures = [x for x in all_events if x['type']['name'] == "Duel"]
    
    for p in pressures:
        attributes = {
            "Player": p['player']['name'],
            "Team": p['team']['name'],
            "Position": p['position']['name'],
            "X": p['location'][0],
            "Y": p['location'][1],
            "Type of Attack": p['play_pattern']['name'],
            "Half": p['period'],
            "MatchID": match_id,
            "CompetitionID": compID
        }
        
        DuelData.append(attributes)

In [33]:
DuelData = pd.DataFrame(data=DuelData, columns=['CompetitionID','MatchID','Team', 'Player', 'Position', 'Type of Attack', 'X', 'Y'])
DuelData.head()
DuelData.to_csv('/Users/jmuwsoccer/Documents/Python/CSV/'+str(competition)+' Duels.csv')

In [34]:
RecovData = []

# template URL at which each game's event data is stored
match_data_url = "https://raw.githubusercontent.com/statsbomb/open-data/master/data/events/{}.json"

# This takes some time, naturally
for match_id in match_ids:
    
    # Get all events from a particular game
    all_events = requests.get(url=match_data_url.format(match_id)).json()
        
    # Filter them by type-name Pass
    pressures = [x for x in all_events if x['type']['name'] == "Ball Recovery"]
    
    for p in pressures:
        attributes = {
            "Player": p['player']['name'],
            "Team": p['team']['name'],
            "Position": p['position']['name'],
            "X": p['location'][0],
            "Y": p['location'][1],
            "Type of Attack": p['play_pattern']['name'],
            "Half": p['period'],
            "MatchID": match_id,
            "CompetitionID": compID
        }
        
        RecovData.append(attributes)

In [35]:
RecovData = pd.DataFrame(data=RecovData, columns=['CompetitionID','MatchID','Team', 'Player', 'Position', 'Type of Attack', 'X', 'Y'])
RecovData.head()
RecovData.to_csv('/Users/jmuwsoccer/Documents/Python/CSV/'+str(competition)+' Recoveries.csv')